In [23]:

# %%
import glob
import pandas as pd
import os
import xarray as xr

# Define campaign name (update as needed)
campaign_name = 'Sanna'  # Replace with the actual campaign name if necessary
#campaign_name = "Forel"
# Define the base path for the data
base_path = fr"C:\Users\cruz\Documents\SENSE\CTD_processing\data\Level1\{campaign_name}"

# Create an empty list to store individual dataframes
def find_all_folders_with_csv(base_path):
    """
    Recursively find all folders containing CSV files
    """
    folders_with_csv = []
    
    for root, dirs, files in os.walk(base_path):
        print(f"Checking folder: {root}")  # Debugging statement
        # Check if there are any CSV files in this directory
        csv_files = [file for file in files if file.lower().endswith('.csv')]
        if csv_files:
            print(f"Found CSV files: {csv_files}")  # Debugging statement
            folders_with_csv.append(root)
            
    return folders_with_csv
# Find all folders containing CSV files
folderlist = find_all_folders_with_csv(base_path)

print(f"Found {len(folderlist)} folders with CSV files:")
for folder in folderlist:
    print(f"- {folder}")

Checking folder: C:\Users\cruz\Documents\SENSE\CTD_processing\data\Level1\Sanna
Checking folder: C:\Users\cruz\Documents\SENSE\CTD_processing\data\Level1\Sanna\Idronaut
Found CSV files: ['20240711_1412_idronaut_profile_1.csv', '20240711_1957_idronaut_profile_1.csv', '20240712_0949_idronaut_profile_1.csv', '20240712_1517_idronaut_profile_1.csv', '20240713_0749_idronaut_profile_1.csv', '20240713_1459_idronaut_profile_1.csv', '20240714_0805_idronaut_profile_1.csv', '20240714_1513_idronaut_profile_1.csv', '20240714_1811_idronaut_profile_1.csv', '20240714_2125_idronaut_profile_1.csv', '20240715_0517_idronaut_profile_1.csv', '20240715_0845_idronaut_profile_1.csv', '20240715_2158_idronaut_profile_1.csv', '20240716_0750_idronaut_profile_1.csv', '20240716_1441_idronaut_profile_1.csv', '20240716_2048_idronaut_profile_1.csv', '20240717_0747_idronaut_profile_1.csv', '20240717_1348_idronaut_profile_1.csv', '20240718_0751_idronaut_profile_1.csv', '20240718_1337_idronaut_profile_1.csv', '20240718_170

In [21]:

dfs = []
# Process files in each subfolder
for subfolder_path in folderlist:
    # Find all CSV and NetCDF files in the subfolder
    csv_files = glob.glob(os.path.join(subfolder_path, "*.csv"))
    nc_files = glob.glob(os.path.join(subfolder_path, "*.nc"))
    files = csv_files + nc_files

    # Filter out files containing "metadata" in their names
    files = [file for file in files if ("metadata" not in os.path.basename(file).lower())]

    # Process each file
    for file in files:
        try:
            # Extract the filename without extension
            filename = os.path.basename(file)
            filename_without_ext = os.path.splitext(filename)[0]
            file_ext = os.path.splitext(file)[1].lower()

            if file_ext == '.csv':
                # Read the CSV file
                df = pd.read_csv(file)
            else:
                print(f"Unsupported file format: {file}")
                continue

            # Add metadata columns
            df['profile_name'] = filename_without_ext
            #df['subfolder'] = os.path.basename(subfolder_path)

            # Append to the list
            dfs.append(df)
            print(f"Successfully read: {filename}")
        except Exception as e:
            print(f"Error reading {file}: {e}")

# Combine all dataframes into a single dataframe
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"Combined {len(dfs)} files into a dataframe with {combined_df.shape[0]} rows and {combined_df.shape[1]} columns")

    # Save the combined dataframe to a new CSV file
    output_path = r"C:\Users\cruz\Documents\SENSE\CTD_processing\data\combined" + f"/combined_{campaign_name}_L1.csv"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Ensure the output directory exists
    combined_df.to_csv(output_path, index=False)
    print(f"Saved combined data to: {output_path}")
else:
    print("No files were successfully read.")

Successfully read: 20240703_1346_idronaut_profile_1.csv
Successfully read: 20240703_1346_idronaut_profile_2.csv
Successfully read: 20240706_1548_idronaut_profile_1.csv
Successfully read: 20240707_1658_idronaut_profile_1.csv
Successfully read: 20240707_1658_idronaut_profile_2.csv
Successfully read: 20240707_1658_idronaut_profile_3.csv
Successfully read: 20240707_1658_idronaut_profile_4.csv
Successfully read: 20240707_1658_idronaut_profile_5.csv
Successfully read: 20240708_1337_idronaut_profile_1.csv
Successfully read: 2024-07-03T050545 SBE0251267_profile_1.csv
Successfully read: 2024-07-04T024636 SBE0251267_profile_1.csv
Successfully read: 2024-07-05T025055 SBE0251267_profile_1.csv
Successfully read: 2024-07-06T053446 SBE0251267_profile_1.csv
Successfully read: 2024-07-06T053446 SBE0251267_profile_2.csv
Successfully read: 2024-07-07T041252 SBE0251267_profile_1.csv
Successfully read: 2024-07-07T084335 SBE0251267_profile_1.csv
Successfully read: 2024-07-07T094429 SBE0251267_profile_1.csv


In [22]:
combined_df["datetime"]

0         2024-03-07 13:47:27.350
1         2024-03-07 13:47:27.590
2         2024-03-07 13:47:27.840
3         2024-03-07 13:47:28.090
4         2024-03-07 13:47:28.330
                   ...           
251203                        NaN
251204                        NaN
251205                        NaN
251206                        NaN
251207                        NaN
Name: datetime, Length: 251208, dtype: object